# Lab 6 - Security Data Science

Andrés Estuardo Montoya Wilhelm

In [5]:
from scapy.all import *
from scapy.layers.inet import IP, TCP, UDP
import pandas as pd

## Preamble

In [3]:
def scapy_test():
    print("Starting packet capture...")
    
    #capture 25 packets
    packets = sniff(count=25)
    
    print("\nVariable type:", type(packets))
    print("Number of packets captured:", len(packets))
    print("\nFirst packet type:", type(packets[0]))
    
    #first 5 packets
    print("\nContent of first 5 packets:")
    for i in range(min(5, len(packets))):
        print(f"\nPacket {i+1}:")
        print(packets[i].show())

In [4]:
scapy_test()

Starting packet capture...

Variable type: <class 'scapy.plist.PacketList'>
Number of packets captured: 25

First packet type: <class 'scapy.layers.l2.Ether'>

Content of first 5 packets:

Packet 1:
###[ Ethernet ]### 
  dst       = 60:38:e0:a8:80:3a
  src       = 4e:e4:0d:79:80:25
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 192
     id        = 0
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = udp
     chksum    = 0xa41e
     src       = 192.168.5.137
     dst       = 142.250.64.227
     \options   \
###[ UDP ]### 
        sport     = 51533
        dport     = https
        len       = 172
        chksum    = 0x753
###[ Raw ]### 
           load      = 'E\\xe8\x16.\\xef\\xf4UM6٤nL4y\\x89\\xa5\\x85\\xeb\x08\\xa6\\x8f\\xd5-\\xd3.\\x92Ӭ\\xf0\\x88\\xf3r\\xac)\\x9b9\\x82:\\xa3\\x95(\\xd5ɴ\\xc0*\\x8a\\x9b\\x8d\\x9f\x1fM(.\x05ڶ&0\\x86Y2\\xefE}8>h0ۏ\\xc0\\x80h.R?\\xc4\x12\\xf8\\xc8ؼ\\x93MsX\\xe2\\xb4\

## Analysis

In [9]:
def analyze_pcap(file):
    print("Reading PCAP file...")
    packets = rdpcap(file)
    
    # Convert packets to DataFrame
    print("\nConverting packets to DataFrame...")
    packet_list = []
    
    for packet in packets:
        if IP in packet:
            packet_info = {
                'Src Address': packet[IP].src,
                'Dst Address': packet[IP].dst,
                'Src Port': packet[TCP].sport if TCP in packet else packet[UDP].sport if UDP in packet else 'N/A',
                'Dst Port': packet[TCP].dport if TCP in packet else packet[UDP].dport if UDP in packet else 'N/A'
            }
            packet_list.append(packet_info)
    
    df = pd.DataFrame(packet_list)
    
    return df

In [15]:
df = analyze_pcap('./analisis_paquetes.pcap')


Reading PCAP file...

Converting packets to DataFrame...


In [16]:
print(df.head())

   Src Address  Dst Address  Src Port  Dst Port
0   10.1.10.53  84.54.22.33        53        53
1  84.54.22.33   10.1.10.53        53        53
2   10.1.10.53  84.54.22.33        53        53
3  84.54.22.33   10.1.10.53        53        53
4   10.1.10.53  84.54.22.33        53        53


In [13]:
print(df[['Src Address', 'Dst Address', 'Src Port', 'Dst Port']].head())

   Src Address  Dst Address  Src Port  Dst Port
0   10.1.10.53  84.54.22.33        53        53
1  84.54.22.33   10.1.10.53        53        53
2   10.1.10.53  84.54.22.33        53        53
3  84.54.22.33   10.1.10.53        53        53
4   10.1.10.53  84.54.22.33        53        53


### All source IP's

In [17]:
print(df['Src Address'].unique())

['10.1.10.53' '84.54.22.33' '75.75.75.75']


### All destination IP's

In [18]:
print(df['Dst Address'].unique())

['84.54.22.33' '10.1.10.53' '75.75.75.75']


### Most frequent source IP 

In [19]:
most_frequent_src = df['Src Address'].mode().iloc[0]
print(f"IP: {most_frequent_src}")

IP: 10.1.10.53


**Most frequent destination IP**

In [26]:
src_communications = df[df['Src Address'] == most_frequent_src]
most_frequent_dst = src_communications['Dst Address'].mode().iloc[0]
print(most_frequent_dst)

84.54.22.33


**Most frequent destination port**

In [28]:
most_frequent_port = src_communications['Dst Port'].mode().iloc[0]
print(most_frequent_port)

53


**Source ports used**

In [32]:
print(src_communications['Src Port'].unique())

[   53 15812 23903]


## Graphs